# How this notebook works

We run FEAT repeatedly for the first-level analyses (one per run), the second-level analyses (across runs per session per subject), and the third-level analyses (per run, across subjects).

To do so, we create:
1. EV files that FEAT understands for the events of interest and motion parameters (unconvolved)
2. 4D niftis that are brain masked and header-corrected to ensure FSL understands they have a TR of 3
3. Manually create 1 .fsf-file using the FSL GUI (for a single run of a single sub of a single session)
4. Use this .fsf-file as a template, and generate .fsf-files for each unique run
5. Run the first-levels
6. Manually create 1 .fsf-file using the FSL GUI again (for fixed effects analysis of single session)
7. Use this .fsf-file as a template, and generate .fsf-files for each FE analysis
8. Run the second-levels
9. Manually create 1 .fsf-file using the FSL GUI again (for FLAME-1 random effects analysis of all subjects across a single session)
10. Use this .fsf-file as a template, and generate .fsf-files for each FE analysis
11. Run the third-levels

We do this for "sessions": Single echo, multi echo, echo-1, echo-2, echo-3 (the latter three are not sessions but treating them as such is slightly easier for coding)

We repeat the first-levels and second-levels for both FWHM=0 and FWHM=5 mm.

In a folder `./feat_files`, we store every .fsf-file. Using this notebook, we can call the first-levels and third-levels with a multiprocess pool. Weirdly, this does not work for the fixed effects (no idea why), so we generate a batch-script to run those.

All manually created .fsf-files are:
- `./feat_files/sub-01/ses-se/run-1/fwhm-5/design.fsf`
- `./feat_files/sub-01/ses-se_fe/fwhm-5/design.fsf`
- `./feat_files/ses-se_flame1/fwhm-5/cope1/design.fsf`

In [24]:
import pandas as pd
import numpy as np
import glob

# for masking
from nilearn.input_data import NiftiMasker
import nibabel as nib
import itertools
import multiprocessing as mp

import os
import shutil

## 1. Create EV-files

A. ev-files with actual events, 3-column format. Will be convolved, temporal derivative will be included

In [49]:
def make_ev_files(sub, ses, run, evs=('go_trial', 'failed_stop', 'successful_stop')):
    if 'echo' in ses:
        ses_ = 'me'
    else:
        ses_ = ses
        
    fn = './data/deriv/fmriprep/sub-{sub}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-stop_run-{run}_events.tsv'.format(**{'sub':sub, 'ses':ses_, 'run':run})
    output_dir = './data/feat_files/sub-{sub}/ses-{ses}/run-{run}/evs/'.format(**{'sub':sub, 'ses':ses, 'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    events = pd.read_csv(fn, sep='\t')
    
    # remove go/failed stop trials with RT > 1
    # 'response_time' is in time relative to run onset; 'rt' is relative to stimulus onset
    events['rt'] = events['response_time'] - events['onset']
    events = events.loc[~((events['rt'] > 1) & (events['trial_type'] != 'successful_stop'))]

    events['weight'] = 1.
    events['onset'] -= 1.5  # slice timing correction

    for i, ev in enumerate(evs):
        events.loc[events['trial_type']==ev, ['onset', 'duration', 'weight']].to_csv(output_dir+'{}.txt'.format(ev), sep='\t', index=False, header=False)

subjects = np.arange(1, 19)
# sessions = ['se', 'me']
sessions = ['echo-1', 'echo-2', 'echo-3']
runs = [1,2,3]

for sub in subjects:
    for ses in sessions:
        if sub == 12 and not ses == 'se':
            continue
            
        for run in runs:
            if sub == 17 and run == 3:
                continue
            make_ev_files(str(sub).zfill(2), ses, run)

B. ev-files with motion parameters that will not be convolved

In [27]:
def make_motion_ev_files(sub, ses, run, evs=('trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'framewise_displacement')):

    output_dir = './data/feat_files/sub-{sub}/ses-{ses}/run-{run}/evs/'.format(**{'sub':sub, 'ses':ses, 'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    if ses == 'se':
        fn = './data/deriv/fmriprep/sub-{sub}/ses-se/func/sub-{sub}_ses-se_task-stop_run-{run}_desc-confounds_regressors.tsv'.format(**{'sub':sub, 'run':run})
    else:
        fn = './data/deriv/fmriprep/sub-{sub}/ses-me/func/sub-{sub}_ses-me_task-stop_run-{run}_echo-1_desc-confounds_regressors.tsv'.format(**{'sub':sub, 'run':run})

    events = pd.read_csv(fn, sep='\t')

    for i, ev in enumerate(evs):
        events[ev].fillna(0).to_csv(output_dir+'{}.txt'.format(ev), sep='\t', index=False, header=False)
    
subjects = np.arange(1, 19)
# sessions = ['se', 'me']
sessions = ['echo-1', 'echo-2', 'echo-3']
runs = [1,2,3]

for sub in subjects:
    for ses in sessions:
        if sub == 12 and not ses == 'se':
            continue
            
        for run in runs:
            if sub == 17 and run == 3:
                continue
            make_motion_ev_files(str(sub).zfill(2), ses, run)

## 2. Apply brain mask to data & Correct headers
The single echo data has not been masked yet.

For some reason, FSL wrongly believes the func data files have a TR of 1. This needs to be corrected. We can use FSLmerge for this.

In [37]:
def get_mask_fn(sub, ses, run):
    sub_str = str(sub).zfill(2)
    session = 'me' if 'echo' in ses or ses == 'me' else 'se'  # sorry about the confusing variable names
    mask_fn = 'data/deriv/fmriprep/sub-{}/ses-{}/func/sub-{}_ses-{}_task-stop_run-{}_{}space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'.format(sub_str, session, sub_str, session, run, '' if session=='se' else 'echo-1_')
    return mask_fn

def mask_img(img, mask):
    masker = NiftiMasker(mask)

    data_masked = masker.fit_transform(img)
    data_masked_epispace = masker.inverse_transform(data_masked)
    return data_masked_epispace

def mask_data_and_fix_header(arg):
    sub, ses, run = arg
    if sub == 17 and run == 3:
        return 0
    if sub == 12 and not ses == 'se':
        return 0
    
    if ses == 'se':
        session_name = ses
        current_fn = 'sub-{}_ses-se_task-stop_run-{}_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz'.format(str(sub).zfill(2), run)
    elif ses == 'me':
        session_name = ses
        current_fn = 'sub-{}_ses-me_task-stop_run-{}_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz'.format(str(sub).zfill(2), run)
    else:
        echo_n = ses[-1]
        session_name = 'me'
        current_fn = 'sub-{}_ses-me_task-stop_run-{}_echo-{}_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz'.format(str(sub).zfill(2), run, echo_n)
        
    input_dir = './data/deriv/fmriprep/sub-{}/ses-{}/func'.format(str(sub).zfill(2), session_name)
    output_dir = './data/feat_files/sub-{sub}/ses-{ses}/run-{run}/func'.format(**{'sub':str(sub).zfill(2), 
                                                                                  'ses':ses, 
                                                                                  'run':run})
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        
    img_fn = os.path.join(input_dir, current_fn)
    output_fn = os.path.join(output_dir, current_fn)
    mask_fn = get_mask_fn(sub, ses, run)

    print(img_fn)
    masked_data = mask_img(img_fn, mask_fn)
    nib.save(masked_data, output_fn)
    
    # make sure header is OK
    import subprocess
    return_code = subprocess.run(["fslmerge", "-tr", output_fn, output_fn, "3"])

    return 0

subjects = np.arange(1, 19)
sessions = ['se', 'me', 'echo-1', 'echo-2', 'echo-3']
runs = [1,2,3]

to_run = itertools.product(subjects, sessions, runs)
to_run_list = list(to_run)

# sequential:
# for i in to_run_list:
#     mask_data_and_fix_header(i)

# MP:
# with mp.Pool(15) as p:
#     p.map(mask_data_and_fix_header, to_run_list)

./data/deriv/fmriprep/sub-01/ses-se/func/sub-01_ses-se_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz
./data/deriv/fmriprep/sub-01/ses-me/func/sub-01_ses-me_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz
./data/deriv/fmriprep/sub-02/ses-se/func/sub-02_ses-se_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz
./data/deriv/fmriprep/sub-02/ses-me/func/sub-02_ses-me_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz
./data/deriv/fmriprep/sub-04/ses-me/func/sub-04_ses-me_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz
./data/deriv/fmriprep/sub-03/ses-se/func/sub-03_ses-se_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz
./data/deriv/fmriprep/sub-03/ses-me/func/sub-03_ses-me_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz
./data/deriv/fmriprep/sub-05/ses-me/func/sub-05_ses-me_task-stop_run-1_space-MNI152NLin2009cA

/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


./data/deriv/fmriprep/sub-05/ses-se/func/sub-05_ses-se_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz
./data/deriv/fmriprep/sub-04/ses-se/func/sub-04_ses-se_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-pac

./data/deriv/fmriprep/sub-01/ses-me/func/sub-01_ses-me_task-stop_run-2_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz


...

/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


./data/deriv/fmriprep/sub-18/ses-me/func/sub-18_ses-me_task-stop_run-3_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


./data/deriv/fmriprep/sub-18/ses-se/func/sub-18_ses-se_task-stop_run-3_space-MNI152NLin2009cAsym_desc-preproc-hp_bold.nii.gz


/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/stevenm/.conda/envs/python3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


## 3. At this point in the pipeline, we manually create a single .fsf-file in FEAT
All preprocessing steps turned off (except smoothing), and no post-stats.
Afterwards, we make copies of this .fsf-file for each subject & session & run, and change the filenames that the fsf-file points to.

We run with both fwhm = 5 mm (standard GLM) and fwhm = 0 mm (to allow for running LISA)

Also, from here on, we treat echos as 'sessions' for simplicity in coding

## 4. Generate .fsf-files for all first-levels

In [28]:
# orig_fsf = './data/feat_files/sub-01/ses-se/run-1.feat/design.fsf'
# with open(orig_fsf, 'r') as f:
#     txt = f.read()

# function to adapt a design file
def fix_fsf_file(sub, ses, run, fwhm=0,
                 orig_fsf='./data/feat_files/sub-01/ses-se/run-1/fwhm-5/design.fsf', 
                 write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    # replace output directory
    fsf = fsf.replace('set fmri(outputdir) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-01/ses-se/run-1"',
                      'set fmri(outputdir) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}/run-{}"'.format(fwhm, str(sub).zfill(2), ses, run))
    
    # replace directories for func files & ev files
    fsf = fsf.replace("/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-se/run-1",
                      "/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-{}/ses-{}/run-{}".format(str(sub).zfill(2), ses, run))
    
    # replace filename, NB: no extension to fn..
    if ses == 'me':
        fn_to_run = "sub-{sub}_ses-{ses}_task-stop_run-{run}_space-MNI152NLin2009cAsym_desc-preproc-hp-optcomb_bold".format(**{'sub':sub, 'ses':ses, 'run':run})
    elif ses == 'se':
        fn_to_run = "sub-{sub}_ses-{ses}_task-stop_run-{run}_space-MNI152NLin2009cAsym_desc-preproc-hp_bold".format(**{'sub':sub, 'ses':ses, 'run':run})
    elif 'echo-' in ses:
        echo_n = ses[-1]
        fn_to_run = "sub-{sub}_ses-me_task-stop_run-{run}_echo-{echo}_space-MNI152NLin2009cAsym_desc-preproc-hp_bold".format(**{'sub':sub, 'run':run, 'echo': echo_n})
        
    fsf = fsf.replace("sub-01_ses-se_task-stop_run-1_space-MNI152NLin2009cAsym_desc-preproc-hp_bold", fn_to_run)
    
    # replace smoothing fwhm
    fsf = fsf.replace("set fmri(smooth) 5", 
                      "set fmri(smooth) {}".format(int(fwhm)))
    
    # subject 6 ses me run 3: only 206 volumes instead of 343
    if sub == '06' and ses is not 'se' and run == 3:
        fsf = fsf.replace("set fmri(npts) 343", "set fmri(npts) 206")
    
    # save fsf as txt
    if write_out:
        output_dir = './data/feat_files/sub-{}/ses-{}/run-{}/fwhm-{}'.format(sub, ses, run, fwhm)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
# function to run in parallel
def run_feat(fsf_file):
    print(fsf_file)
    import subprocess
    return_code = subprocess.run(["feat", fsf_file])
    print('{}: {}'.format(fsf_file, return_code))
    return return_code

#fix_fsf_file('01', 'se', 2, write_out=False)

In [29]:
# main loop
subjects = np.arange(1, 19)
sessions = ['se', 'me', 'echo-1', 'echo-2', 'echo-3']
runs = [1,2,3]

for fwhm in [5, 0]:
    # generate all fsf files first
    for sub in subjects:
        for ses in sessions:
            if sub == 12 and not ses == 'se':
                continue

            for run in runs:
                if sub == 17 and run == 3:
                    continue
                else:
                    fix_fsf_file(str(sub).zfill(2), ses, run, fwhm=fwhm)
    
# find all newly created design files
all_fsfs = glob.glob('./data/feat_files/sub-*/ses-*/run*/fwhm-*/*.fsf')
all_fsfs.sort()

## 5. Run the first-levels
##### Runtime estimate 

First start (`feat_fwhm-0/sub-01/ses-echo-1/run-1.feat/report.html`): 14:32h
End: 15:33h

- About 1 hour per run on this server (Intel Gold 6134 CPU @ 3.20GHz)
- 3 * 17 * 5 * 2 = (3 runs, 17 participants, 5 data types, 2 fwhms) = 510 runs = 510 CPU hour total
- On the current server, we run 25 processes simultaneously 
- 510cpuh / 25cpu =~ 20 hours in total

In reality, over 24h --> some overhead

In [ ]:
with mp.Pool(28) as p:
    outputs = p.map(run_feat, all_fsfs)

./data/feat_files/sub-02/ses-echo-2/run-3/fwhm-0/design.fsf
./data/feat_files/sub-01/ses-echo-2/run-3/fwhm-0/design.fsf
./data/feat_files/sub-01/ses-echo-1/run-1/fwhm-0/design.fsf
./data/feat_files/sub-01/ses-me/run-2/fwhm-0/design.fsf
./data/feat_files/sub-02/ses-echo-1/run-1/fwhm-0/design.fsf
./data/feat_files/sub-01/ses-echo-3/run-2/fwhm-5/design.fsf
./data/feat_files/sub-02/ses-echo-1/run-3/fwhm-5/design.fsf
./data/feat_files/sub-03/ses-echo-1/run-3/fwhm-5/design.fsf
./data/feat_files/sub-01/ses-se/run-1/fwhm-5/design.fsf
./data/feat_files/sub-01/ses-echo-1/run-3/fwhm-5/design.fsf
./data/feat_files/sub-03/ses-me/run-2/fwhm-0/design.fsf
./data/feat_files/sub-04/ses-echo-1/run-1/fwhm-0/design.fsf
./data/feat_files/sub-03/ses-se/run-1/fwhm-5/design.fsf
./data/feat_files/sub-03/ses-echo-3/run-2/fwhm-5/design.fsf
./data/feat_files/sub-04/ses-echo-1/run-3/fwhm-5/design.fsf
./data/feat_files/sub-04/ses-echo-2/run-3/fwhm-0/design.fsf
./data/feat_files/sub-04/ses-echo-3/run-2/fwhm-5/design.

Minor correction: some .feat-directories accidentally already existed before running feat. Feat then outputs to an +.feat-directory. Here, we find all +.feat directories, and rename them to remove the +

In [77]:
# all_featplus_dirs = glob.glob('./data/feat_fwhm-*/sub*/ses*/run*+.feat/')

# for fn in all_featplus_dirs:
#     if os.path.exists(fn) and os.path.exists(fn.replace('+.feat', '.feat')):
#         fn_no_plus = fn.replace('+.feat/', '.feat')
#         os.rename(fn_no_plus, fn_no_plus.replace('.feat', '_OLD.feat'))
#         os.rename(fn, fn_no_plus)

## Interim: Fix "registration not run"-issue for fixed effects

Feat higher-order analyses require "registration to have been performed", which is true, but not by FSL so FSL doesn't know. To do this, "trick" FSL into thinking they have been run by linking identity matrices to the feat dirs.

See also https://neurostars.org/t/performing-full-glm-analysis-with-fsl-on-the-bold-images-preprocessed-by-fmriprep-without-re-registering-the-data-to-the-mni-space/784

In [32]:
def link_files(feat_dir, fsl_dir='/usr/share/fsl/5.0'):
    reg_dir = os.path.join(feat_dir, 'reg')
    if not os.path.exists(reg_dir):
        os.makedirs(reg_dir)
    os.system('ln -s {}/etc/flirtsch/ident.mat {}/reg/example_func2standard.mat'.format(fsl_dir, feat_dir))
    os.system('ln -s {}/etc/flirtsch/ident.mat {}/reg/standard2example_func.mat'.format(fsl_dir, feat_dir))
    os.system('ln -s {}/mean_func.nii.gz {}/reg/standard.nii.gz'.format(feat_dir, feat_dir))
    return 0

subjects = np.arange(1, 19)
sessions = ['se', 'me', 'echo-1', 'echo-2', 'echo-3']
runs = [1,2,3]

for fwhm in [0, 5]:
    for sub in subjects:
        for ses in sessions:
            if sub == 12 and not ses == 'se':
                continue

            for run in runs:
                if sub == 17 and run == 3:
                    continue

                else:
                    feat_dir = '/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}/run-{}.feat'.format(fwhm, str(sub).zfill(2), ses, run)
                    if os.path.exists(feat_dir):
                        link_files(feat_dir)

## 6. Manually create single fixed-effects .fsf-file here

## 7. Generate .fsf-files for second-level fixed-effects

In [33]:
def fix_fsf_file_fe(sub, ses, fwhm,
                    orig_fsf='./data/feat_files/sub-01/ses-se_fe/fwhm-5/design.fsf', 
                    write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    for run in [1,2,3]:
        # replace directories..
        fsf = fsf.replace("/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-01/ses-se/run-{}".format(run),
                          "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}/run-{}".format(fwhm, str(sub).zfill(2), ses, run))
    
    # output directory
    fsf = fsf.replace("/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-01/ses-se_fe", 
                      "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}_fe".format(fwhm, str(sub).zfill(2), ses, run))
    
    # For subject 17 and 6, remove run 3
    if sub == '17' or sub == '06':
        # change n inputs
        fsf = fsf.replace("set fmri(npts) 3", "set fmri(npts) 2")
        fsf = fsf.replace("set fmri(multiple) 3", "set fmri(multiple) 2")
        
        # change feat directories
        fsf = fsf.replace('\n# 4D AVW data or FEAT directory (3)', '')
        fsf = fsf.replace('\nset feat_files(3) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}/run-3.feat"'.format(fwhm, sub, ses), '')
        
        # change EV specification
        fsf = fsf.replace('\n# Higher-level EV value for EV 1 and input 3', '')
        fsf = fsf.replace('\nset fmri(evg3.1) 1.0', '')
        fsf = fsf.replace('\n# Group membership for input 3', '')
        fsf = fsf.replace('\nset fmri(groupmem.3) 1', '')
        
    # save fsf as txt
    if write_out:
        output_dir = './data/feat_files/sub-{}/ses-{}_fe/fwhm-{}'.format(str(sub).zfill(2), ses, fwhm)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
#fix_fsf_file_fe('01', 'me', write_out=False)

subjects = np.arange(1, 19)
sessions = ['se', 'me', 'echo-1', 'echo-2', 'echo-3']

for fwhm in [0, 5]:
    for sub in subjects:
        for ses in sessions:
            if sub == 12 and not ses == 'se':
                continue
            else:
                fix_fsf_file_fe(str(sub).zfill(2), ses, fwhm=fwhm, write_out=True)

For some bloody reason this won't run from within a jupyter notebook, so make a batch files that can be run from within the system

I made 10 .sh files that can be run in parallel. Each one makes 7 serial .feat-calls.
The outer wrapper "run_fe_batch.sh" calls the 10 .sh-files in parallel


Runtime estimate: an hour or so?

In [34]:
all_fsfs = glob.glob('/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-*/ses-*_fe/fwhm-*/design.fsf')
all_fsfs.sort()
all_fsfs

['/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-echo-1_fe/fwhm-0/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-echo-1_fe/fwhm-5/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-echo-2_fe/fwhm-0/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-echo-2_fe/fwhm-5/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-echo-3_fe/fwhm-0/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-echo-3_fe/fwhm-5/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-me_fe/fwhm-0/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-me_fe/fwhm-5/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-se_fe/fwhm-0/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-01/ses-se_fe/fwhm-5/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-02/ses-echo-1_fe/fwhm-0/design.fsf',
 '/home/stevenm/MultiEchoEPISeq/data/

In [35]:
all_fsfs = glob.glob('/home/stevenm/MultiEchoEPISeq/data/feat_files/sub-*/ses-*_fe/fwhm-*/design.fsf')
all_fsfs.sort()

total_n_calls = len(all_fsfs)  # 70 in total
n_files = 25  # this is the number of parallel processes that will run

# evenly divide number of feat calls over n_files
n_rows_per_file = np.floor(total_n_calls / n_files)
remainder = total_n_calls % n_files
rows_per_file = np.repeat(n_rows_per_file, n_files)
for i in range(remainder):
    rows_per_file[i] += 1

# ensure we're not missing one
assert np.sum(rows_per_file) == total_n_calls

txt_files = []
# loop over files
for txt_file_n, n_lines_this_file in enumerate(rows_per_file.astype(int)):
    this_txt = []
    for row in range(n_lines_this_file):
        this_fsf_file = all_fsfs.pop(0)
        this_txt.append('feat {}\n'.format(this_fsf_file))
    this_txt.insert(0, '#/bin/bash\n')
    
    this_fn = "/home/stevenm/MultiEchoEPISeq/data/feat_files/fe_batch-{}.sh".format(txt_file_n)
    with open(this_fn, 'w') as f:
        f.writelines(this_txt)
    os.system('chmod +x {}'.format(this_fn))
    txt_files.append(this_fn + ' &\n')

# write outer wrapper for a single command line call
txt_files.insert(0, '#/bin/bash\n')
with open('./data/feat_files/run_fe_batch.sh', 'w') as f:
    f.writelines(txt_files)
os.system('chmod +x ./data/feat_files/run_fe_batch.sh')

assert len(all_fsfs) == 0

print("Call this line: \n/home/stevenm/MultiEchoEPISeq/data/feat_files/run_fe_batch.sh > /home/stevenm/MultiEchoEPISeq/data/feat_files/fixed_effects_outputs.txt")

Call this line: 
/home/stevenm/MultiEchoEPISeq/data/feat_files/run_fe_batch.sh > /home/stevenm/MultiEchoEPISeq/data/feat_files/fixed_effects_outputs.txt


## 8. Call the line above

### Fixed effects done

## 9. Create .fsf-file for third-level random effects

Only for FWHM = 5 mm

Use Flame1

5x9 models (2 sessions + 3 echos, 9 copes) = another whopping 45 FLAME1-calls

In [138]:
# useful call for pasting paths
# txt = ['/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-{}/ses-se_fe.gfeat/cope1.feat'.format(str(x).zfill(2)) for x in np.arange(1, 19) if not x == 12]
# print('\n'.join(txt))

## 10. Generate other third-level .fsf-files

In [36]:
def fix_fsf_file_flame1(ses, fwhm, cope,
                        orig_fsf='./data/feat_files/ses-se_flame1/fwhm-5/cope1/design.fsf', 
                        write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    # output directory
    fsf = fsf.replace("/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/third_levels/ses-se/cope1",
                      "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/third_levels/ses-{}/cope{}".format(fwhm, ses, cope))
    
    for sub in np.arange(1, 19):
        fsf = fsf.replace('set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-{}/ses-se_fe.gfeat/cope1.feat"'.format(sub, str(sub).zfill(2)),
            'set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}_fe.gfeat/cope{}.feat"'.format(sub, fwhm, str(sub).zfill(2), ses, cope))
        
        # feat files 12-17 correspond to subjects 13-18; 12 was removed
        fsf = fsf.replace('set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-{}/ses-se_fe.gfeat/cope1.feat"'.format(sub-1, str(sub).zfill(2)),
            'set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}_fe.gfeat/cope{}.feat"'.format(sub-1, fwhm, str(sub).zfill(2), ses, cope))
        
    # save fsf as txt
    if write_out:
        output_dir = './data/feat_files/ses-{}_flame1/fwhm-{}/cope{}'.format(ses, fwhm, cope)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
#fix_fsf_file_fe('01', 'me', write_out=False)

sessions = ['se', 'me', 'echo-1', 'echo-2', 'echo-3']
runs = [1,2,3]
copes = np.arange(1, 10)

for fwhm in [5]:  # don't run unsmoothed - that was only for LISA
    for cope in copes:
        for ses in sessions:
            fix_fsf_file_flame1(ses, fwhm=fwhm, cope=cope, write_out=True)

In [37]:
# try to run (does this one work from command line or do we need to do this stupid batching again?)
# !feat ./data/feat_fwhm-5/ses-se_flame1/cope1/design.fsf

# ok this seems to work again!

In [38]:
# no need to run any fwhm=0 second levels
all_fsfs = glob.glob('./data/feat_files/ses-*_flame1/fwhm-5/cope*/design.fsf')
all_fsfs

['./data/feat_files/ses-echo-2_flame1/fwhm-5/cope6/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope4/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope2/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope7/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope3/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope5/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope9/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope1/design.fsf',
 './data/feat_files/ses-echo-2_flame1/fwhm-5/cope8/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope6/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope4/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope2/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope7/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope3/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope5/design.fsf',
 './data/feat_files/ses-se_flame1/fwhm-5/cope9/design.fsf',
 './

In [39]:
all_fsfs = glob.glob('/home/stevenm/MultiEchoEPISeq/data/feat_files/ses-*_flame1/fwhm-5/cope*/design.fsf')
all_fsfs.sort()

total_n_calls = len(all_fsfs)  # 70 in total
n_files = 15  # this is the number of parallel processes that will run

# evenly divide number of feat calls over n_files
n_rows_per_file = np.floor(total_n_calls / n_files)
remainder = total_n_calls % n_files
rows_per_file = np.repeat(n_rows_per_file, n_files)
for i in range(remainder):
    rows_per_file[i] += 1

# ensure we're not missing one
assert np.sum(rows_per_file) == total_n_calls

txt_files = []
# loop over files
for txt_file_n, n_lines_this_file in enumerate(rows_per_file.astype(int)):
    this_txt = []
    for row in range(n_lines_this_file):
        this_fsf_file = all_fsfs.pop(0)
        this_txt.append('feat {}\n'.format(this_fsf_file))
    this_txt.insert(0, '#/bin/bash\n')
    
    this_fn = "/home/stevenm/MultiEchoEPISeq/data/feat_files/re_batch-{}.sh".format(txt_file_n)
    with open(this_fn, 'w') as f:
        f.writelines(this_txt)
    os.system('chmod +x {}'.format(this_fn))
    txt_files.append(this_fn + ' &\n')

# write outer wrapper for a single command line call
txt_files.insert(0, '#/bin/bash\n')
with open('./data/feat_files/run_re_batch.sh', 'w') as f:
    f.writelines(txt_files)
os.system('chmod +x ./data/feat_files/run_re_batch.sh')

assert len(all_fsfs) == 0

print("Call this line: \n/home/stevenm/MultiEchoEPISeq/data/feat_files/run_re_batch.sh > /home/stevenm/MultiEchoEPISeq/data/feat_files/random_effects_outputs.txt")

Call this line: 
/home/stevenm/MultiEchoEPISeq/data/feat_files/run_re_batch.sh > /home/stevenm/MultiEchoEPISeq/data/feat_files/random_effects_outputs.txt


## 11. Run third-levels

In [ ]:
# with mp.Pool(25) as p:
#     outputs = p.map(run_feat, all_fsfs)

./data/feat_files/ses-echo-2_flame1/fwhm-5/cope6/design.fsf
./data/feat_files/ses-echo-2_flame1/fwhm-5/cope1/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope3/design.fsf
./data/feat_files/ses-echo-2_flame1/fwhm-5/cope2/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope2/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope1/design.fsf
./data/feat_files/ses-echo-2_flame1/fwhm-5/cope8/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope4/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope9/design.fsf
./data/feat_files/ses-echo-1_flame1/fwhm-5/cope4/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope6/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope7/design.fsf
./data/feat_files/ses-echo-2_flame1/fwhm-5/cope4/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope5/design.fsf
./data/feat_files/ses-se_flame1/fwhm-5/cope8/design.fsf
./data/feat_files/ses-echo-1_flame1/fwhm-5/cope9/design.fsf
./data/feat_files/ses-echo-1_flame1/fwhm-5/cope6/design.fsf
./data/feat_file

# FLAME1 done

That's all folks?

In [1]:
!ls ./data/feat_fwhm-0/sub-01/ses-se_fe.gfeat/cope4.feat/stats/

cope1.nii.gz			 res4d.nii.gz	  weights1.nii.gz
dof				 smoothness	  zflame1lowertstat1.nii.gz
logfile				 tdof_t1.nii.gz   zflame1uppertstat1.nii.gz
mean_random_effects_var1.nii.gz  tstat1.nii.gz	  zstat1.nii.gz
pe1.nii.gz			 varcope1.nii.gz


In [40]:
def fix_fsf_file_flame12(ses, fwhm, cope,
                        orig_fsf='./data/feat_files/ses-se_flame12/fwhm-5/cope1/design.fsf', 
                        write_out=True):
    # read fsf as txt
    with open(orig_fsf, 'r') as f:
        fsf = f.read()
    
    # output directory
    fsf = fsf.replace("/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/third_levels/ses-se/cope1",
                      "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/third_levels/ses-{}/cope{}".format(fwhm, ses, cope))
    
    for sub in np.arange(1, 19):
        fsf = fsf.replace('set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-{}/ses-se_fe.gfeat/cope1.feat"'.format(sub, str(sub).zfill(2)),
            'set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}_fe.gfeat/cope{}.feat"'.format(sub, fwhm, str(sub).zfill(2), ses, cope))
        
        # feat files 12-17 correspond to subjects 13-18; 12 was removed
        fsf = fsf.replace('set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-5/sub-{}/ses-se_fe.gfeat/cope1.feat"'.format(sub-1, str(sub).zfill(2)),
            'set feat_files({}) "/home/stevenm/MultiEchoEPISeq/data/feat_fwhm-{}/sub-{}/ses-{}_fe.gfeat/cope{}.feat"'.format(sub-1, fwhm, str(sub).zfill(2), ses, cope))
        
    # save fsf as txt
    if write_out:
        output_dir = './data/feat_files/ses-{}_flame12/fwhm-{}/cope{}'.format(ses, fwhm, cope)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(os.path.join(output_dir, 'design.fsf'), 'w') as f:
            f.write(fsf)
    else:
        return fsf
    
#fix_fsf_file_fe('01', 'me', write_out=False)

sessions = ['se', 'me']
runs = [1,2,3]
copes = np.arange(1, 4)

for fwhm in [5]:  # don't run unsmoothed - that was only for LISA
    for cope in copes:
        for ses in sessions:
            fix_fsf_file_flame12(ses, fwhm=fwhm, cope=cope, write_out=True)

In [41]:
# no need to run any fwhm=0 second levels
all_fsfs = glob.glob('./data/feat_files/ses-*_flame12/fwhm-5/cope*/design.fsf')
all_fsfs

['./data/feat_files/ses-se_flame12/fwhm-5/cope2/design.fsf',
 './data/feat_files/ses-se_flame12/fwhm-5/cope3/design.fsf',
 './data/feat_files/ses-se_flame12/fwhm-5/cope1/design.fsf',
 './data/feat_files/ses-me_flame12/fwhm-5/cope2/design.fsf',
 './data/feat_files/ses-me_flame12/fwhm-5/cope3/design.fsf',
 './data/feat_files/ses-me_flame12/fwhm-5/cope1/design.fsf']

In [43]:
all_fsfs = glob.glob('/home/stevenm/MultiEchoEPISeq/data/feat_files/ses-*_flame12/fwhm-5/cope*/design.fsf')
all_fsfs.sort()

total_n_calls = len(all_fsfs)  # 70 in total
n_files = 6  # this is the number of parallel processes that will run

# evenly divide number of feat calls over n_files
n_rows_per_file = np.floor(total_n_calls / n_files)
remainder = total_n_calls % n_files
rows_per_file = np.repeat(n_rows_per_file, n_files)
for i in range(remainder):
    rows_per_file[i] += 1

# ensure we're not missing one
assert np.sum(rows_per_file) == total_n_calls

txt_files = []
# loop over files
for txt_file_n, n_lines_this_file in enumerate(rows_per_file.astype(int)):
    this_txt = []
    for row in range(n_lines_this_file):
        this_fsf_file = all_fsfs.pop(0)
        this_txt.append('feat {}\n'.format(this_fsf_file))
    this_txt.insert(0, '#/bin/bash\n')
    
    this_fn = "/home/stevenm/MultiEchoEPISeq/data/feat_files/re_batch-{}.sh".format(txt_file_n)
    with open(this_fn, 'w') as f:
        f.writelines(this_txt)
    os.system('chmod +x {}'.format(this_fn))
    txt_files.append(this_fn + ' &\n')

# write outer wrapper for a single command line call
txt_files.insert(0, '#/bin/bash\n')
with open('./data/feat_files/run_re_batch.sh', 'w') as f:
    f.writelines(txt_files)
os.system('chmod +x ./data/feat_files/run_re_batch.sh')

assert len(all_fsfs) == 0

print("Call this line: \n/home/stevenm/MultiEchoEPISeq/data/feat_files/run_re_batch.sh > /home/stevenm/MultiEchoEPISeq/data/feat_files/random_effects_outputs.txt")

Call this line: 
/home/stevenm/MultiEchoEPISeq/data/feat_files/run_re_batch.sh > /home/stevenm/MultiEchoEPISeq/data/feat_files/random_effects_outputs.txt


In [44]:
import subprocess
import os
import glob
import numpy as np
import pandas
import nilearn
import nibabel as nib

class SLM(object):
    def __init__(self, copes, session='se',
                 feat_root = './data/feat_fwhm-0',
                 save_dir='./data/feat_fwhm-0/third_levels', 
                 tmp_dir = '/home/stevenm/MultiEchoEPISeq/lisa_tmp_dir',
                 level = 0.05):
        self.copes = copes
        self.session = session
        self.lisa_tmp_dir = tmp_dir
        self.feat_root = feat_root
        self.save_dir = save_dir + '/ses-{}'.format(self.session)
        self.level = level
    
    def _get_first_levels(self, cope):
        
        first_levels = glob.glob(os.path.join(self.feat_root, 
                                                   'sub-*', 
                                                   'ses-{}_fe.gfeat'.format(self.session), 
                                                   'cope{}.feat'.format(cope),
                                                   'stats/cope1.nii.gz'))
#         first_levels = [x.replace('./data/', '') for x in first_levels]
        return first_levels
        
    def _load_if_exists(self, fn):
        path = os.path.join(self.save_dir, fn)
        if os.path.exists(path):
            return nib.load(path)
        else:
            return None
        
    def _save_contrast_nii(self, nii, fn):
        # save second level models
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)
        
        nib.save(nii, os.path.join(self.save_dir, fn))

    def fit_lisa(self):
        # empty contrasts dictionary, will point to filenames
#         self.first_level_contrasts = {x: [] for x in self.contrasts.keys()}
        
        if not os.path.exists(self.lisa_tmp_dir):
            os.makedirs(self.lisa_tmp_dir)
    
        for cope in self.copes:  #contrast_name, contrast in self.contrasts.items():
            if os.path.exists(os.path.join(self.lisa_tmp_dir, 'ses-{}_cope{}.nii'.format(self.session, cope))):
                print('already exists, skipping')
                continue
                
            # try to load previously run first...
            contrast_fn = 'cope-{}.nii.gz'.format(cope)
            contrast_nii = self._load_if_exists(contrast_fn)
            if contrast_nii is not None:
                self.zmaps[contrast_name] = contrast_nii
                continue
            
            first_level_fns = self._get_first_levels(cope=cope)

#             print(first_level_fns)
            print('Converting to float32 using fslmaths...')
            fns_full = [os.path.join(self.lisa_tmp_dir, x.replace(self.feat_root+'/', '').replace('/', '_')) for x in first_level_fns]
            for fn_in, fn_tmp in zip(first_level_fns, fns_full):
                cmd = ['fslmaths', fn_in, fn_tmp,  '-odt', 'float']
                print(' '.join(cmd))
                subprocess.call(cmd)
            
            cmd_onesample = self._make_lipsia_command(fns=fns_full)
            cmd_conversion = self._make_lipsia_vnifti_command(fn='/data/ses-{}_cope{}'.format(self.session, cope))
            print('Running command: {}'.format(cmd_onesample))
            print(' '.join(cmd_onesample))
            lipsia_output_onesample = subprocess.call(cmd_onesample)
            print('done, output: {}'.format(lipsia_output_onesample))
            
            print('Running command: {}'.format(cmd_conversion))
            lipsia_output_conversion = subprocess.call(cmd_conversion)
            print('done, output: {}'.format(lipsia_output_conversion))
            
            # cleanup
            print('cleanup...')
            [os.remove(x) for x in glob.glob(os.path.join(self.lisa_tmp_dir, '*.nii.gz'))]
            os.remove(os.path.join(self.lisa_tmp_dir, 'second_level_contrast.v'))
            
#             [os.remove(os.path.join(self.lisa_tmp_dir, x)) for x in first_level_fns]
#             os.remove(os.path.join(self.lisa_tmp_dir, 'second_level_contrast.v'))
            
#             self.zmaps[contrast_name] = nib.load(os.path.join(self.lisa_tmp_dir, 'second_level_contrast.nii'))
#             self._save_contrast_nii(self.zmaps[contrast_name], contrast_fn)
            
    def _make_lipsia_command(self, fns, out_fn='/data/second_level_contrast.v', docker=True):

        if docker:
            #fns = ['/data/' + x for x in fns]  # make sure to point at the /data-directory inside the Docker container
            fns = [x.replace(self.lisa_tmp_dir, '/data') for x in fns]
            cmd = ['docker', 'run', '-v', self.lisa_tmp_dir + ':/data', 
                   'lipsia', 'vlisa_onesample', '-alpha', str(self.level), '-in']
            cmd += fns
            cmd += ['-out', out_fn, '-cleanup', 'false']
            #cmd = ' '.join(cmd)
        else:
            # not implemented
            pass
        
        return cmd
    
    def _make_lipsia_vnifti_command(self, fn='/data/cope{}', docker=True):
        
        if docker:
            cmd = ['docker', 'run', '-v', self.lisa_tmp_dir + ':/data', 
                   'lipsia', 'vnifti', '-in', '/data/second_level_contrast' + '.v', '-out', fn + '.nii']
#            cmd = ' '.join(cmd)
        else:
            # not implemented
            pass
        
        return cmd
            

In [45]:
for session in ['se', 'me']:
    slm = SLM(copes=[1,2,3,7,8,9], session=session, level=0.025)  # NB: two-sided test, so half the level
    slm.fit_lisa()

already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping
already exists, skipping


In [ ]:
## done